In [ ]:
#Bert lib
!pip install transformers
#usefull in cleaning the data
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from collections import OrderedDict
import torch
from torch import nn
from operator import itemgetter
import nltk
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
import logging
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

class STS_model(nn.Module): 
    def __init__(self, model_path): 
      super(STS_model, self).__init__() 

      # Load model from HuggingFace Hub
      self.tokenizer = AutoTokenizer.from_pretrained(model_path)
      self.Bert_representations = AutoModel.from_pretrained(model_path)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        
    def forward(self, model_input): 
      # Tokenize sentences
      encoded_input = self.tokenizer(model_input, padding=True, truncation=True, return_tensors='pt')

      model_output = self.Bert_representations(**encoded_input)

      # Perform pooling. In this case, max pooling.
      sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

      return sentence_embeddings

In [ ]:
# model = STS_model('sentence-transformers/bert-base-nli-mean-tokens')
model = STS_model('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
sentences = ['This is an example sentence', 'Each sentence is converted']
output = model(sentences)
print(output)

In [ ]:
from torch import nn
triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
anchor = torch.randn(100, 128, requires_grad=True)
positive = torch.randn(100, 128, requires_grad=True)
negative = torch.randn(100, 128, requires_grad=True)
output = triplet_loss(anchor, positive, negative)
output.backward()
print(output)

In [ ]:
inputs = [["I want to eat","lets eat","how are you?"],["how much are glasses", "at what price are those glasses", "Hello"], ["Hello", "how are you?", "I am hungry"]]

In [ ]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.train().to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
for input in inputs:
        output = model(input)
        anchor = output[0]
        positive = output[1]
        negative = output[2]
        loss = triplet_loss(anchor, positive, negative)
        print(loss)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()